# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset, WeightedRandomSampler
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
from torch.cuda.amp import GradScaler

In [ ]:
# model architecture
class ConvNet(nn.Module):

    def __init__(self):
        
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1, 3), stride=(1,1), padding=0)
        self.bn1   = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.2)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(1, 3), stride=(1,1), padding=0)
        self.bn2   = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(p=0.2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1, 5), stride=(1,1), padding=0)
        self.bn3   = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 5), stride=(1, 5))
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(p=0.2)
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=(1, 7), stride=(1,1), padding=0)
        self.bn4   = nn.BatchNorm2d(96)
        self.pool4 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(p=0.2)
        
        self.conv5 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(5, 5), stride=(1,1), padding=0)
        self.bn5   = nn.BatchNorm2d(128)
        self.pool5 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.relu5 = nn.ReLU()
        self.drop5 = nn.Dropout(p=0.2)
        
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1,1), padding=0)
        self.bn6   = nn.BatchNorm2d(256)
        self.pool6 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 1))
        self.relu6 = nn.ReLU()
        self.drop6 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(in_features=int(3*256), out_features=128)
        self.relu7 = nn.ReLU()
        
        self.fc2 = nn.Linear(in_features=128, out_features=32)
        self.relu8 = nn.ReLU()
        
        self.fc3 = nn.Linear(in_features=32, out_features=2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = x.unsqueeze(1)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool3(x)
        x = self.relu3(x)
        x = self.drop3(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool4(x)
        x = self.relu4(x)
        x = self.drop4(x)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool5(x)
        x = self.relu5(x)
        x = self.drop5(x)
        
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool6(x)
        x = self.relu6(x)
        x = self.drop6(x)
    
        x = x.reshape(-1, int(3*256))
        x = self.fc1(x)
        x = self.relu7(x)
        
        x = self.fc2(x)
        x = self.relu8(x)
        
        x = self.fc3(x)
        x = self.sigmoid(x)
  
        return x

In [ ]:
# initialize model
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print("device type: ", device)

train_idx = np.load('train_index_file')
test_idx = np.load('test_index_file')

model = ConvNet()
model.to(device)

criterion = nn.BCELoss()
batch_size = 32
num_epochs = 200

scaler = GradScaler() # mixed precision training
optimizer = optim.Adam(model.parameters(), lr=0.001)
transform = transforms.Compose([transforms.ToTensor()])
dataset = MyDataset() # call custom dataset function

train_sampler = SubsetRandomSampler(train_idx)
train_loader = DataLoader(dataset, batch_size = batch_size, sampler = train_sampler)

test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(dataset, batch_size = batch_size, sampler = test_sampler, shuffle = False)

In [ ]:
# training and testing loops

# local log
train_local_log = {"loss": [], "accuracy": []}
test_local_log = {"loss": [], "accuracy": []}

# early stopping
best_test_loss = float('inf')
epochs_since_improvement = 0

for epoch in range(num_epochs):

    print(f"\nEpoch {epoch+1} is starting... please wait!")
    start_time = time.time()

    # train the model
    train_loss = 0.0
    train_correct = 0

    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        target_onehot = F.one_hot(targets, num_classes=2).float()

        if torch.backends.cudnn.enabled:
            torch.backends.cudnn.benchmark = True

        output = model(inputs)
        loss = criterion(output,target_onehot)

        model.zero_grad()
        loss.backward()

        optimizer.step()

        # save train loss for this batch  
        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(output.data, 1)            
        train_correct += (predicted == targets).sum(dim=0).item() 

    # save average train loss and acc
    train_loss = train_loss / len(train_loader.sampler)
    train_accuracy = train_correct / len(train_loader.sampler) * 100
    train_local_log['loss'].append(train_loss)
    train_local_log['accuracy'].append(train_accuracy)

    # test the model
    test_loss = 0.0
    test_correct = 0
    val_preds = []
    val_labels = []

    model.eval()
    for inputs, targets in test_loader:

        inputs,targets = inputs.to(device),targets.to(device)
        target_onehot = F.one_hot(targets, num_classes=2).float()
        output = model(inputs)
        loss = criterion(output,target_onehot) 
        # save test loss for this batch  
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(output.data, 1)
        val_preds.extend(predicted.cpu().numpy())
        val_labels.extend(targets.cpu().numpy())
        test_correct += (predicted == targets).sum(dim=0).item()

    # save average test loss and acc
    test_loss = test_loss / len(test_loader.sampler)

    # early stopping
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement >= 10:
        print(f'Validation loss has not improved for {epochs_since_improvement} epochs. Stopping training...')
        break
    else:
        pass

    test_accuracy = test_correct / len(test_loader.sampler) * 100
    test_local_log['loss'].append(test_loss)
    test_local_log['accuracy'].append(test_accuracy)
    print("Epoch:{}/{} \t AVG Training Loss:{:.3f} \t AVG Test Loss:{:.3f} \t AVG Training Acc {:.2f} % \t AVG Test Acc {:.2f} %".format(epoch+1, num_epochs, train_loss, test_loss, train_accuracy, test_accuracy))

# save the model weights
PATH = "the model weights address"
torch.save(model.state_dict(), PATH)

# Calibration

In [ ]:
# initiate model for fine-tuning

perf_metrics_log = {"accuracy": [], "sensitivity": [], "specificity": [], "cm": []}
roc_log = {"fpr": [],"tpr": [],"thresholds": [],"roc_auc": []}
train_log = []
test_log = []

# load calibration dataset
train_idx = np.load('train_index_file')
test_idx = np.load('test_index_file')

model2 = ConvNet()
model2.load_state_dict(torch.load(PATH)) # load weights
model2.to(device)

criterion = nn.BCELoss()
batch_size = 128
num_epochs = 200

scaler = GradScaler() # mixed precision training
optimizer = optim.Adam(model2.parameters(), lr=0.000001)
transform = transforms.Compose([transforms.ToTensor()])
dataset = MyDataset() # call custom dataset function

train_sampler = SubsetRandomSampler(train_idx)
train_loader = DataLoader(dataset, batch_size = batch_size, sampler = train_sampler)

test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(dataset, batch_size = batch_size, sampler = test_sampler, shuffle = False)

In [ ]:
# training and testing loops

# local performance metrics log
train_local_log = {"loss": [], "accuracy": []}
test_local_log = {"loss": [], "accuracy": []}

# early stopping
best_test_loss = float('inf')
epochs_since_improvement = 0

for epoch in range(num_epochs):

    print(f"\nEpoch {epoch+1} is starting... please wait!")
    start_time = time.time()

    # train the model
    train_loss = 0.0
    train_correct = 0

    model2.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        target_onehot = F.one_hot(targets, num_classes=2).float()
    
        if torch.backends.cudnn.enabled:
            torch.backends.cudnn.benchmark = True

        output = model2(inputs)
        loss = criterion(output,target_onehot)
    
        # backward pass
        model2.zero_grad()
        loss.backward()

        optimizer.step()

        # save train loss for this batch  
        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(output.data, 1)            
        train_correct += (predicted == targets).sum(dim=0).item() 

    # save average train loss and acc
    train_loss = train_loss / len(train_loader.sampler)
    train_accuracy = train_correct / len(train_loader.sampler) * 100
    train_local_log['loss'].append(train_loss)
    train_local_log['accuracy'].append(train_accuracy)

    # test the model
    test_loss = 0.0
    test_correct = 0
    val_preds = []
    val_labels = []
    val_probs = []

    model2.eval()
    for inputs, targets in test_loader:

        inputs,targets = inputs.to(device),targets.to(device)
        target_onehot = F.one_hot(targets, num_classes=2).float()
        output = model2(inputs)
        probs = torch.sigmoid(output) 
        val_probs.extend(probs[:, 1].cpu().detach().numpy())
        loss = criterion(output,target_onehot) 
        # save test loss for this batch  
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(probs.data, 1)
        val_preds.extend(predicted.cpu().numpy())
        val_labels.extend(targets.cpu().numpy())
        test_correct += (predicted == targets).sum(dim=0).item()

    # save average test loss and acc
    test_loss = test_loss / len(test_loader.sampler)

    # early stopping
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement >= 10:
        print(f'Validation loss has not improved for {epochs_since_improvement} epochs. Stopping training...')
        break
    else:
        pass

    test_accuracy = test_correct / len(test_loader.sampler) * 100
    test_local_log['loss'].append(test_loss)
    test_local_log['accuracy'].append(test_accuracy)
    print("Epoch:{}/{} \t AVG Training Loss:{:.3f} \t AVG Test Loss:{:.3f} \t AVG Training Acc {:.2f} % \t AVG Test Acc {:.2f} %".format(epoch+1, num_epochs, train_loss, test_loss, train_accuracy, test_accuracy))

train_log.append(train_local_log)
test_log.append(test_local_log)

# performance metrics log
val_labels = np.array(val_labels)
val_preds = np.array(val_preds)

cm = confusion_matrix(val_labels, val_preds)
perf_metrics_log["cm"].append(cm)

tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
perf_metrics_log["accuracy"].append(accuracy*100)
perf_metrics_log["sensitivity"].append(sensitivity*100)
perf_metrics_log["specificity"].append(specificity*100)
print(f"accuracy : {accuracy*100:.2f} %")
print(f"sensitivity : {sensitivity*100:.2f} %")
print(f"specificity : {specificity*100:.2f} %")

In [ ]:
# ROC log
fpr, tpr, thresholds = roc_curve(val_labels, val_probs)
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()